# QKV Attentionについて

$$Attention(Q, K, V) = softmax(\frac{QK^T}{\sqrt{d_k}})V$$

Attnetionは上式のように定義されている.

これはさまざまな解釈ができるが僕の好みはトークン(特徴量ベクトル)の集合をその集合内の類似性を考慮した新しい集合に進化させるためのものだと考えることである.

データセットが与えられたとき, そのデータセット内の類似性を求めようと思ったとき, もっとも素朴なやり方はデータセット同士の行列積を求めることである.
これはデータ同士の内積はcos類似度に対応することからのアナロジー取れ, またグラム行列からも理解できる.

Attentionはその考えたデータセットを一列に並べて, (データ数(=バッチサイズ)*特徴量)のサイズの大きさの行列とし, そしてデータセットの類似性を行列積として計算し, さらにそれをデータセットに適用して進化させる. 
Attentionはそれを
1. まずデータセットをQuery(類似度用), Key(類似度用), Value(出力用)の3つに加工する
2. QueryとKeyの行列積を求め, 特徴量次元を考慮して正規化を行うことで類似度を作る(Attention weight)
3. QueryとKeyで作った類似度を使って, Valueに適用させて, 新しいデータセットを作る

なぜQueryとKeyの二つがあるかというといろいろ解釈があるが, 自分同士の類似度だけじゃなくて別のデータセットとの類似度を求めるための自由度を持たせたと考えることが僕の好みである.

注意として, Attention自体はほぼ行列積そのものであり, パラメータを持たない.
Attentionのパラメータと呼ばれるものはQuery, Key, Valueを作るときにデータセットにLinearを通すが, このLinearのパラメータのことだ.
(MultiHeadAttentionのときはこういう呼ばれ方がされることがある)


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

def QKVattention(Query, Key, Value, attention_QKV_dim):
    attention_score = torch.matmul(Query, Key.transpose(-2, -1))/attention_QKV_dim
    attention_weight = F.softmax(attention_score, dim=-1)
    attention = torch.matmul(attention_weight, Value)
    return attention

class multihead_attention(nn.Module):
    def __init__(self, Q_tensor_dim, K_tensor_dim, V_tensor_dim, attention_QKV_dim) -> None:
        super().__init__()
        self.Q_tensor_dim = Q_tensor_dim
        self.K_tensor_dim = K_tensor_dim
        self.V_tensor_dim = V_tensor_dim
        self.attention_QKV_dim = attention_QKV_dim
        self.linear_Q = nn.Linear(Q_tensor_dim, attention_QKV_dim)
        self.linear_K = nn.Linear(K_tensor_dim, attention_QKV_dim)
        self.linear_V = nn.Linear(V_tensor_dim, attention_QKV_dim)

    def forward(self, Q_tensor, K_tensor, V_tensor):
        Q = self.linear_Q(Q_tensor)
        K = self.linear_K(K_tensor)
        V = self.linear_V(V_tensor)
        attention_matrix = QKVattention(Q, K, V, self.attention_QKV_dim)
        return attention_matrix

In [5]:
Q_tensor = torch.rand(4, 5)
K_tensor = torch.rand(4, 5)
V_tensor = torch.rand(4, 5)
print(Q_tensor)


tensor([[0.1742, 0.2942, 0.2527, 0.8415, 0.9068],
        [0.0389, 0.4320, 0.1214, 0.8945, 0.4234],
        [0.4047, 0.2564, 0.5341, 0.1083, 0.9343],
        [0.6210, 0.2592, 0.4510, 0.9350, 0.7226]])


In [6]:
attention = multihead_attention(5, 5, 5, 3)
print(attention(Q_tensor, K_tensor, V_tensor))

tensor([[0.1745, 0.1961, 0.5204],
        [0.1746, 0.1960, 0.5204],
        [0.1749, 0.1961, 0.5206],
        [0.1746, 0.1956, 0.5201]], grad_fn=<MmBackward0>)


In [4]:
class transformer_block(nn.Module):
    def __init__(self, attention_QKV_dim) -> None:
        super().__init__()
        self.attn_layer = multihead_attention(attention_QKV_dim, attention_QKV_dim, attention_QKV_dim, attention_QKV_dim)
        self.MLP_layer = nn.Sequential(nn.Linear(attention_QKV_dim, attention_QKV_dim), nn.ReLU(), nn.Linear(attention_QKV_dim, attention_QKV_dim))
        
    def forward(self, Q_tensor, K_tensor, V_tensor):
        attention_matrix = self.attn_layer(Q_tensor, K_tensor, V_tensor)
        after_mlp = self.MLP_layer(attention_matrix)
        return after_mlp
        
class GenerativePretrainedTransformer(nn.Module):
    def __init__(self, Q_tensor_dim, K_tensor_dim, V_tensor_dim, attention_QKV_dim, transformer_layer_number):
        super().__init__()
        self.initial_attention = multihead_attention(Q_tensor_dim, K_tensor_dim, V_tensor_dim, attention_QKV_dim)
        self.multi_layer_transformer_blocks = nn.ModuleList([transformer_block(attention_QKV_dim) for i in range(transformer_layer_number)])
        
    def forward(self, Q_tensor, K_tensor, V_tensor):
        initial_attn = self.initial_attention(Q_tensor, K_tensor, V_tensor)
        for idx, block in enumerate(self.multi_layer_transformer_blocks):
            if idx == 0:
                out = block(initial_attn, initial_attn, initial_attn)
            else:
                out = block(out, out, out)
        return out